In [9]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from sklearn.model_selection import train_test_split
import numpy as np

# Create a sample dataset (you can replace this with your actual data)
class CustomDataset(Dataset):
    def __init__(self, x, y):
        self.x = x
        self.y = y

    def __len__(self):
        return len(self.x)

    def __getitem__(self, idx):
        return self.x[idx], self.y[idx]

# Define a CNN model with a fully connected layer for classification
class CNNClassifier(nn.Module):
    def __init__(self, input_size, output_size):
        super(CNNClassifier, self).__init__()
        self.fc1 = nn.Linear(input_size, 512)
        self.fc2 = nn.Linear(512, 256)
        self.relu = nn.ReLU()
        self.fc3 = nn.Linear(256, output_size)  # Adjust the input size based on your data

    def forward(self, x):
        x = self.fc1(x)
        x = self.relu(x)
        x = self.fc2(x)
        x = self.relu(x)
        x = self.fc3(x)
        x = self.relu(x)
        return x

# Define hyperparameters
input_size = 90 # Input size: 31 sequences of length 3
output_size = 90  # Number of classes
learning_rate = 0.0001
batch_size = 512
num_epochs = 2000

# Generate sample data (replace with your actual data)
# For this example, we'll generate random data
x = np.load('x2.npy')
print(x.shape)
# print(np.max(x))
# x = x + np.random.uniform(-0.5*np.max(x), 0.5*np.max(x))
x = np.squeeze(x).reshape(x.shape[0], -1).astype('float32') 
y = np.load('y2.npy').astype('float32')

# Split data into training and testing sets
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=42)

# Create data loaders
train_dataset = CustomDataset(x_train, y_train)
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)

# Initialize the model and optimizer
model = CNNClassifier(input_size, output_size)
criterion = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)
loss_list = []
# Training loop
for epoch in range(num_epochs):
    for inputs, targets in train_loader:
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, targets)
        loss.backward()
        optimizer.step()
    loss_list.append(loss.item())
    print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item()}')
np.save('loss.npy', loss_list)
torch.save(model.state_dict(), 'slagent.pth')




(14049, 90)
Epoch [1/2000], Loss: 0.2841489315032959
Epoch [2/2000], Loss: 0.15604540705680847
Epoch [3/2000], Loss: 0.14774572849273682
Epoch [4/2000], Loss: 0.14071542024612427
Epoch [5/2000], Loss: 0.1372620165348053
Epoch [6/2000], Loss: 0.13383570313453674
Epoch [7/2000], Loss: 0.13207127153873444
Epoch [8/2000], Loss: 0.1307225078344345
Epoch [9/2000], Loss: 0.12994158267974854
Epoch [10/2000], Loss: 0.125080406665802
Epoch [11/2000], Loss: 0.11917029321193695
Epoch [12/2000], Loss: 0.11458457261323929
Epoch [13/2000], Loss: 0.11450575292110443
Epoch [14/2000], Loss: 0.10933826118707657
Epoch [15/2000], Loss: 0.10743985325098038
Epoch [16/2000], Loss: 0.10510720312595367
Epoch [17/2000], Loss: 0.1019827350974083
Epoch [18/2000], Loss: 0.10222525894641876
Epoch [19/2000], Loss: 0.10216447710990906
Epoch [20/2000], Loss: 0.0997270867228508
Epoch [21/2000], Loss: 0.09887872636318207
Epoch [22/2000], Loss: 0.0991050973534584
Epoch [23/2000], Loss: 0.09917671978473663
Epoch [24/2000],

In [10]:
# Test the model
test_dataset = CustomDataset(x_test, y_test)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=True)

model.eval()
model.load_state_dict(torch.load('slagent.pth'))

correct = 0
total = 0
with torch.no_grad():
    for inputs, targets in test_loader:
        outputs = model(inputs).numpy()
        outputs = np.rint(outputs).astype(int)
        
        # print('out: ', outputs[5])
        # print('target: ', targets.numpy()[5])
        # print(targets.shape[0])
        total += targets.shape[0]* targets.shape[1]
        for i, outvector in enumerate(outputs):
            for j, out in enumerate(outvector):
            # print(out)
                # print(targets[i, j].numpy())
                if out == int(targets[i, j].numpy()):
                    correct += 1

accuracy = 100 * correct / total
print(f'Test Accuracy: {accuracy}%')

Test Accuracy: 94.58046658758403%


In [3]:
outputs.shape

(250, 90)

In [9]:
x.shape

(1310, 90)